In [ ]:
import pandas as pd
import numpy as np
pd.set_option('max_columns', 100)

import matplotlib as mpl
import matplotlib.pyplot as plt

default_colors = plt.rcParams['axes.prop_cycle'].by_key()['color']


In [ ]:
mpl.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (8.0, 6.0)

mpl.rcParams.update({'font.size': 16})
mpl.rc('axes', linewidth=2)
mpl.rcParams.update({'xtick.minor.visible':True})
mpl.rcParams.update({'ytick.minor.visible':True})
mpl.rcParams.update({'xtick.major.size'   : 10 })
mpl.rcParams.update({'xtick.minor.size'   : 5  })
mpl.rcParams.update({'xtick.major.width'  : 1.5})
mpl.rcParams.update({'xtick.minor.width'  : 1  })
mpl.rcParams.update({'ytick.major.size'   : 10 })
mpl.rcParams.update({'ytick.minor.size'   : 5  })
mpl.rcParams.update({'ytick.major.width'  : 1.5})
mpl.rcParams.update({'ytick.minor.width'  : 1  })
mpl.rcParams.update({'ytick.direction'  : 'in'  })
mpl.rcParams.update({'xtick.direction'  : 'in'  })
mpl.rcParams.update({'xtick.top'  : True  })
mpl.rcParams.update({'ytick.right'  : True  })


In [ ]:
# read in first 147 rows
im = pd.read_csv('Downloads/Interview_Tracking_interview_metrics.csv', nrows = 147)


In [ ]:
im


In [ ]:
# rows that are entirely null
im[im.isnull().all(axis=1)]


In [ ]:
# replace data engineer, data analytics engineer with DAE
im['Position'] = im['Position'].str.replace('Data (Analytics )*Engineer', 'DAE', regex = True).str.rstrip(' ')
# get mean and count by position
im_grouped = im.groupby('Position').agg(['mean', 'count'])
im_grouped


In [ ]:
# rows with no data (all mean cols null)
no_data = im_grouped[im_grouped[[col for col in im_grouped.columns if 'mean' in col]].isnull().all(axis=1)]
sorted([row for row in no_data.index])


In [ ]:
# rows with any data
some_data = im_grouped[im_grouped[[col for col in im_grouped.columns if 'mean' in col]].notnull().any(axis=1)]
some_data = some_data.sort_values('Position')
# weighted mean
for row in some_data.index:
    some_data.loc[row, ('Summary', 'mean')] = np.average(
        some_data.loc[row, [col for col in im_grouped.columns if 'mean' in col]].fillna(0),
        weights = some_data.loc[row, [col for col in im_grouped.columns if 'count' in col]]
    )
# count
some_data[('Summary', 'count')] = some_data[[col for col in im_grouped.columns if 'count' in col]].apply(
    sum, axis = 1)
some_data['color'] = default_colors[:some_data.shape[0]]
some_data


In [ ]:
# max count for plotting
count_max = some_data[[col for col in im_grouped.columns if 'count' in col]].max().max()
count_max


In [ ]:
# overall max count for plotting
overall_count_max = some_data[('Summary', 'count')].max()
overall_count_max


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:2]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax1 = plt.subplots()
    sortd['count'].plot.bar(color = sortd['color'])
    ax1.set_ylabel('count (bar)')
    if col == 'Summary':
        ax1.set_ylim([0, overall_count_max + 2])
    else:
        ax1.set_ylim([0, count_max + 2])

    plt.xticks(rotation = 20, ha = 'right')
    plt.title(col)

    ax2 = ax1.twinx()
    ax2.plot(sortd['mean'], linewidth = 2.0, color = 'black')
    ax2.set_ylabel('mean score (line)')
    ax2.set_ylim([0.9, 4.1])

    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    excluded_str = ', '.join(excluded)
    if excluded != []:
        if len(excluded_str) > 60:
            excluded_str = excluded_str.replace(', ', """,
            """)
            ax2.text(-0.35, 3.55, f"No data: {excluded_str}")
        else:     
            ax2.text(-0.35, 3.85, f"No data: {excluded_str}")                


In [ ]:
it = pd.read_csv('Downloads/Interview_Tracking_background.csv', nrows = 148)


In [ ]:
it


In [ ]:
it[it.values  == "ATSV"]


In [ ]:
it[it.isnull().all(axis=1)]


In [ ]:
it = it.dropna(how = 'all')


In [ ]:
it = it.rename(columns = {'Move on to in-person interview?': 'In Person?'})
it['Position'] = it['Position'].str.replace('Data (Analytics )*Engineer', 'DAE', regex = True).str.rstrip(' ')

numeric_cols = ['In Person?', 'Offer?', 'Offer Accepted?']
for col in numeric_cols:
    it[col] = it[col].replace(['No', 'Yes', 'ATSV'], ['0', '1', '1']).astype(float)

it


In [ ]:
for col in it.columns:
    print(it[col].value_counts())
        

In [ ]:
it_grouped_pos_num = it.groupby('Position').agg({col: ['mean', 'count'] for col in numeric_cols})
it_grouped_pos_num


In [ ]:
no_data = it_grouped_pos_num[it_grouped_pos_num[[
    col for col in it_grouped_pos_num.columns if 'mean' in col]].isnull().all(axis=1)]
sorted([row for row in no_data.index])


In [ ]:
some_data = it_grouped_pos_num[it_grouped_pos_num[[
    col for col in it_grouped_pos_num.columns if 'mean' in col]].notnull().any(axis=1)]
some_data = some_data.sort_values('Position')
some_data


In [ ]:
it_consolidated = it.copy()

fields_to_consolidate = {
    'Junior DAE.*': 'Junior DAE',
    '\A(Junior |)Data Scientist.*': 'Junior Data Scientist',
    '.*Data Science Manager': 'Data Science Manager',
    'Senior Data Scientist.*': 'Senior Data Scientist',
    '.*Product Intelligence Consultant.*': 'Product Intelligence Consultant'
}
for field in fields_to_consolidate.keys():
    it_consolidated['Position'] = it_consolidated['Position'].str.replace(
        field, fields_to_consolidate[field])

it_consolidated['General Position'] = it_consolidated['Position']

fields_to_double_consolidate = {
    '\A\w* \w*(\w* \w*)* Manager': 'Manager',
    '.* Senior Manager': 'Senior Manager',
    '.*Consultant': 'Consultant',
    'Intern.*': 'Intern'
}
for field in fields_to_double_consolidate.keys():
    it_consolidated['General Position'] = it_consolidated['General Position'].str.replace(
        field, fields_to_double_consolidate[field])
    
it_consolidated


In [ ]:
for col in numeric_cols:
    plt.figure()
    plt.hist(it_consolidated[col])
    plt.title(col)


In [ ]:
it_consolidated_pos_num = it_consolidated.groupby('Position').agg(
    {col: ['mean', 'count', 'sum'] for col in numeric_cols})
it_consolidated_pos_num


In [ ]:
some_data = it_consolidated_pos_num[it_consolidated_pos_num[[
    col for col in it_consolidated_pos_num.columns if 'mean' in col]].notnull().any(axis=1)]
some_data = some_data.sort_values('Position')
some_data['color'] = (default_colors * 2)[:some_data.shape[0]]
some_data


In [ ]:
count_max = some_data[[col for col in it_consolidated_pos_num.columns if 'count' in col]].max().max()
count_max


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax1 = plt.subplots(figsize = (15, 10))
    sortd['count'].plot.bar(color = sortd['color'])
    ax1.set_ylabel('Sample size (bar)')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)

    ax2 = ax1.twinx()
    ax2.plot(sortd['mean'] * 100, linewidth = 2.0, color = 'black')
    ax2.set_ylabel('Percentage of Candidates (line)', fontsize = 16)
    ax2.set_ylim([-10, 110])

    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax2.text(9, 95, f"No data: {', '.join(excluded)}")
      

In [ ]:
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax1 = plt.subplots(figsize = (15, 10))
for i, col in enumerate(count_cols):
    ax1.bar(sortd.index, sortd[col], color = sortd['color'], alpha = 0.5 + i * 0.25)
ax1.set_ylabel('Sample size (bar)')

plt.xticks(rotation = 45, ha = 'right')
plt.title('Interview Progression')

ax2 = ax1.twinx()
colors = ['red', 'blue', 'green']
for i, col in enumerate(mean_cols):
    ax2.plot(sortd[col] * 100, linewidth = 2.0, color = colors[i])
ax2.set_ylabel('Percentage of Candidates (line)')
ax2.set_ylim([-10, 135])

handles2, labels2 = ax2.get_legend_handles_labels()
patches = [mpl.patches.Patch(color = 'black', alpha = 0.25 + i * 0.25, label = l)
           for i, l in enumerate(labels2)]
ax2.legend(patches + handles2, [''] * 3 + [label[2:-10] for label in labels2], 
           loc = 'upper right', ncol = 2, handletextpad=0.4, columnspacing=0.4)

excluded = list(some_data.index)
for row in sortd.index:
    excluded.remove(row)
if excluded != []:
    ax2.text(9.5, 105, f"No data: {', '.join(excluded)}")


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax = plt.subplots(figsize = (15, 10))
    
    ax.bar(sortd.index, sortd['count'], color = sortd['color'], alpha = 0.5)
    ax.bar(sortd.index, sortd['sum'], color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)
    
    patches = [mpl.patches.Patch(color = 'black', alpha = 0.5, label = 'All'),
               mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes')]
    ax.legend(handles = patches, loc = 'best', handletextpad=0.4, columnspacing=0.4)


    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax.text(9.4, 5.4, f"No data: {', '.join(excluded)}")
      

In [ ]:
cols = [col[0] for col in some_data.columns[:-1:3]]
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax = plt.subplots(figsize = (15, 10))

width = 0.25
ind = np.arange(len(sortd.index))

patches = []
for i, col in enumerate(cols):
    ax.bar(ind + width * i, sortd[(col, 'count')], width, color = sortd['color'], alpha = 0.25 + 0.25 * i)
    ax.bar(ind + width * i, sortd[(col, 'sum')], width, color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')
    
    patches.append(mpl.patches.Patch(color = 'black', alpha = 0.25 + 0.25 * i, label = f'{col} All'))

patches.append(mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes'))
patches.append(mpl.patches.Patch(color = 'white'))

ax.set_xticklabels(sortd.index)
ax.legend(handles = patches, loc = 'best', handletextpad = 0.4, columnspacing = 0.4, ncol = 2)

plt.xticks(np.arange(min(ind) + 0.25, max(ind) + 1.25), rotation = 45, ha = 'right')
plt.title('Interview Progression')


In [ ]:
it_consolidated_gp_num = it_consolidated.groupby('General Position').agg(
    {col: ['mean', 'count', 'sum'] for col in numeric_cols})
it_consolidated_gp_num


In [ ]:
some_data = it_consolidated_gp_num[it_consolidated_gp_num[[
    col for col in it_consolidated_gp_num.columns if 'mean' in col]].notnull().any(axis=1)]
some_data = some_data.sort_values('General Position')
some_data['color'] = (default_colors * 2)[:some_data.shape[0]]
some_data


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax1 = plt.subplots(figsize = (15, 10))
    sortd['count'].plot.bar(color = sortd['color'])
    ax1.set_ylabel('Sample size (bar)')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)

    ax2 = ax1.twinx()
    ax2.plot(sortd['mean'] * 100, linewidth = 2.0, color = 'black')
    ax2.set_ylabel('Percentage of Candidates (line)', fontsize = 16)
    ax2.set_ylim([-10, 110])

    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax2.text(4, 95, f"No data: {', '.join(excluded)}")
      

In [ ]:
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax1 = plt.subplots(figsize = (15, 10))
for i, col in enumerate(count_cols):
    ax1.bar(sortd.index, sortd[col], color = sortd['color'], alpha = 0.5 + i * 0.25)
ax1.set_ylabel('Sample size (bar)')

plt.xticks(rotation = 45, ha = 'right')
plt.title('Interview Progression')

ax2 = ax1.twinx()
colors = ['red', 'blue', 'green']
for i, col in enumerate(mean_cols):
    ax2.plot(sortd[col] * 100, linewidth = 2.0, color = colors[i])
ax2.set_ylabel('Percentage of Candidates (line)')
ax2.set_ylim([-10, 135])

handles2, labels2 = ax2.get_legend_handles_labels()
patches = [mpl.patches.Patch(color = 'black', alpha = 0.25 + i * 0.25, label = l)
           for i, l in enumerate(labels2)]
ax2.legend(patches + handles2, [''] * 3 + [label[2:-10] for label in labels2], 
           loc = 'upper right', ncol = 2, handletextpad=0.4, columnspacing=0.4)

excluded = list(some_data.index)
for row in sortd.index:
    excluded.remove(row)
if excluded != []:
    ax2.text(9.5, 105, f"No data: {', '.join(excluded)}")


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax = plt.subplots(figsize = (15, 10))
    
    ax.bar(sortd.index, sortd['count'], color = sortd['color'], alpha = 0.5)
    ax.bar(sortd.index, sortd['sum'], color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)
    
    patches = [mpl.patches.Patch(color = 'black', alpha = 0.5, label = 'All'),
               mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes')]
    ax.legend(handles = patches, loc = 'best', handletextpad=0.4, columnspacing=0.4)


    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax.text(4.5, 5.4, f"No data: {', '.join(excluded)}")
      

In [ ]:
cols = [col[0] for col in some_data.columns[:-1:3]]
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax = plt.subplots(figsize = (15, 10))

width = 0.25
ind = np.arange(len(sortd.index))

patches = []
for i, col in enumerate(cols):
    ax.bar(ind + width * i, sortd[(col, 'count')], width, color = sortd['color'], alpha = 0.25 + 0.25 * i)
    ax.bar(ind + width * i, sortd[(col, 'sum')], width, color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')
    
    patches.append(mpl.patches.Patch(color = 'black', alpha = 0.25 + 0.25 * i, label = f'{col} All'))

patches.append(mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes'))
patches.append(mpl.patches.Patch(color = 'white'))

ax.set_xticklabels(sortd.index)
ax.legend(handles = patches, loc = 'best', handletextpad = 0.4, columnspacing = 0.4, ncol = 2)

plt.xticks(np.arange(min(ind) + 0.25, max(ind) + 1.25), rotation = 45, ha = 'right')
plt.title('Interview Progression')


In [ ]:
it_consolidated['education'] = np.where(
    it_consolidated['Ph.D'].notnull(), 'PhD', np.where(
        it_consolidated['Masters'].notnull(), 'Masters', np.where(
            it_consolidated['Bachelors'].notnull(), 'Bachelors', 'Unknown'
)))

degree_types = ['Bachelors', 'Masters', 'Ph.D']
for degree in degree_types:
    it_consolidated[degree] = it_consolidated[degree].apply(lambda x: x.split(', ') if type(x) == str else x)

it_consolidated['degrees'] = it_consolidated[degree_types].values.tolist()
it_consolidated['degrees'] = it_consolidated['degrees'].apply(lambda x: [y[0] for y in x if type(y) == list])
it_consolidated['degrees'] = it_consolidated['degrees'].apply(lambda x: [np.nan] if x == [] else x)


In [ ]:
it_consolidated


In [ ]:
it_consolidated[it_consolidated['education'] == 'Unknown']


In [ ]:
it_consolidated['education'].value_counts().reindex(['Unknown', 'Bachelors', 'Masters', 'PhD']).plot(kind = 'bar')


In [ ]:
it_consolidated_edu_num = it_consolidated.groupby('education').agg(
    {col: ['mean', 'count', 'sum'] for col in numeric_cols})
it_consolidated_edu_num


In [ ]:
some_data = it_consolidated_edu_num[it_consolidated_edu_num[[
    col for col in it_consolidated_edu_num.columns if 'mean' in col]].notnull().any(axis=1)]
some_data = some_data.sort_values('education')
some_data['color'] = (default_colors * 2)[:some_data.shape[0]]
some_data


In [ ]:
count_max = some_data[[col for col in it_consolidated_edu_num.columns if 'count' in col]].max().max()
count_max


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax1 = plt.subplots(figsize = (15, 10))
    sortd['count'].plot.bar(color = sortd['color'])
    ax1.set_ylabel('Sample size (bar)')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)

    ax2 = ax1.twinx()
    ax2.plot(sortd['mean'] * 100, linewidth = 2.0, color = 'black')
    ax2.set_ylabel('Percentage of Candidates (line)', fontsize = 16)
    ax2.set_ylim([-10, 110])

    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax2.text(9, 95, f"No data: {', '.join(excluded)}")
      

In [ ]:
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax1 = plt.subplots(figsize = (15, 10))
for i, col in enumerate(count_cols):
    ax1.bar(sortd.index, sortd[col], color = sortd['color'], alpha = 0.5 + i * 0.25)
ax1.set_ylabel('Sample size (bar)')

plt.xticks(rotation = 45, ha = 'right')
plt.title('Interview Progression')

ax2 = ax1.twinx()
colors = ['red', 'blue', 'green']
for i, col in enumerate(mean_cols):
    ax2.plot(sortd[col] * 100, linewidth = 2.0, color = colors[i])
ax2.set_ylabel('Percentage of Candidates (line)')
ax2.set_ylim([-10, 135])

handles2, labels2 = ax2.get_legend_handles_labels()
patches = [mpl.patches.Patch(color = 'black', alpha = 0.25 + i * 0.25, label = l)
           for i, l in enumerate(labels2)]
ax2.legend(patches + handles2, [''] * 3 + [label[2:-10] for label in labels2], 
           loc = 'upper right', ncol = 2, handletextpad=0.4, columnspacing=0.4)

excluded = list(some_data.index)
for row in sortd.index:
    excluded.remove(row)
if excluded != []:
    ax2.text(9.5, 105, f"No data: {', '.join(excluded)}")


In [ ]:
for col in [col[0] for col in some_data.columns[:-1:3]]:
    sortd = some_data.sort_values([(col, 'mean'), (col, 'count')])[col].dropna(subset = ['mean'])
    sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']
    
    fig, ax = plt.subplots(figsize = (15, 10))
    
    ax.bar(sortd.index, sortd['count'], color = sortd['color'], alpha = 0.5)
    ax.bar(sortd.index, sortd['sum'], color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')

    plt.xticks(rotation = 45, ha = 'right')
    plt.title(col)
    
    patches = [mpl.patches.Patch(color = 'black', alpha = 0.5, label = 'All'),
               mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes')]
    ax.legend(handles = patches, loc = 'best', handletextpad=0.4, columnspacing=0.4)


    excluded = list(some_data.index)
    for row in sortd.index:
        excluded.remove(row)
    if excluded != []:
        ax.text(9.4, 5.4, f"No data: {', '.join(excluded)}")
      

In [ ]:
cols = [col[0] for col in some_data.columns[:-1:3]]
mean_cols = [col for col in some_data.columns[:-1:3]]
count_cols = [col for col in some_data.columns[1::3]]

sortd = some_data.sort_values(mean_cols + count_cols)
sortd['color'] = some_data.loc[[row for row in sortd.index], 'color']

fig, ax = plt.subplots(figsize = (15, 10))

width = 0.25
ind = np.arange(len(sortd.index))

patches = []
for i, col in enumerate(cols):
    ax.bar(ind + width * i, sortd[(col, 'count')], width, color = sortd['color'], alpha = 0.25 + 0.25 * i)
    ax.bar(ind + width * i, sortd[(col, 'sum')], width, color = 'None', edgecolor = 'black')
    ax.set_ylabel('Count')
    
    patches.append(mpl.patches.Patch(color = 'black', alpha = 0.25 + 0.25 * i, label = f'{col} All'))

patches.append(mpl.patches.Patch(facecolor = 'white', edgecolor = 'black', label = 'Yes'))
patches.append(mpl.patches.Patch(color = 'white'))

ax.set_xticklabels(sortd.index)
ax.legend(handles = patches, loc = 'best', handletextpad = 0.4, columnspacing = 0.4, ncol = 2)

plt.xticks(np.arange(min(ind) + 0.25, max(ind) + 1.25), rotation = 45, ha = 'right')
plt.title('Interview Progression')


In [ ]:
df.where(df % 3 == 0, -df)